<a href="https://colab.research.google.com/github/DianaMCV/Redes/blob/main/Red_de_comercializaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Pquetería
import pandas as pd
import re
import networkx as nx
#import graph_tool.all as gt
import numpy as np
#import geopandas as gpd  
import matplotlib.pyplot as plt  

In [250]:
#Bases de datos
url='https://raw.githubusercontent.com/DianaMCV/Redes/main/comercio_2019.csv'
url1='https://raw.githubusercontent.com/DianaMCV/Redes/main/comercio_2020.csv'
url2='https://raw.githubusercontent.com/DianaMCV/Redes/main/comercio_2021.csv'
url3='https://raw.githubusercontent.com/RBESPINOSA/proyecto_redes/main/mercados.csv'
url4='https://raw.githubusercontent.com/RBESPINOSA/proyecto_redes/main/distancias_final.csv'

c2019 = pd.read_csv(url, index_col=0, encoding='latin-1') 
c2020 = pd.read_csv(url1, index_col=0, encoding='latin-1') 
c2021 = pd.read_csv(url2, index_col=0, encoding='latin-1') 
cord = pd.read_csv(url3, encoding='latin-1') 
distancias = pd.read_csv(url4, encoding='latin-1') 

ANÁLISIS DE LA RED DE COMERCIALIZACIÓN DE FRUTAS Y HORTALIZAS, 2019

In [251]:
cord.head()

,id,central_abasto,estado,ciudad,coordenadas
0,1,Aguascalientes: Centro Comercial Agropecuario ...,Aguascalientes,Aguascalientes,"21.91350375264395, -102.29531720470844"
1,2,"Baja California : Central de Abasto INDIA, Tij...",Baja California,Tijuana,"32.49474400026095, -116.94692024909172"
2,3,Baja California Sur: Unión de Comerciantes de ...,Baja California Sur,La Paz,"24.1271092193078, -110.3307038550387"
3,4,"Campeche: Mercado ""Pedro Sáinz de Baranda"", Ca...",Campeche,Campeche,"19.84188527351779, -90.53320367292051"
4,5,Chiapas: Central de Abasto de Tuxtla Gutiérrez,Chiapas,Tuxtla Gutiérrez,"16.74342132425459, -93.06736080161691"


In [252]:
def get_x_y(coordenadas, what='x'):
    r = re.search('(\-?\d+\.?\d+), (\-?\d+\.?\d+)', coordenadas)
    if what == 'x':
        return float(r.group(1))
    if what == 'y':
        return float(r.group(2))

In [253]:
coordenadas = pd.DataFrame()
coordenadas['id'] = cord['id']
coordenadas['x'] = cord['coordenadas'].apply(get_x_y)
coordenadas['y'] = cord['coordenadas'].apply(get_x_y, what='y')
coordenadas.head()

,id,x,y
0,1,21.913504,-102.295317
1,2,32.494744,-116.946920
2,3,24.127109,-110.330704
3,4,19.841885,-90.533204
4,5,16.743421,-93.067361


In [267]:
coordenadas.to_csv('coordenadas.csv', index = False)

In [254]:
c2019.columns = c2019.columns.str.lower()
c2019['origen']=c2019['origen'].str.lower()
c2019['destino']=c2019['destino'].str.lower()

c2019.head()

,producto,calidad,presentación,origen,destino,precio mín,precio max,precio frec,obs.
Fecha,,,,,,,,,
22/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit: mercado de abasto 'adolfo lópez mateo...,808,810,809,NaN
23/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit: mercado de abasto 'adolfo lópez mateo...,808,810,809,NaN
24/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit: mercado de abasto 'adolfo lópez mateo...,808,810,809,NaN
25/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit: mercado de abasto 'adolfo lópez mateo...,808,810,809,NaN
26/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit: mercado de abasto 'adolfo lópez mateo...,808,810,809,NaN


In [255]:
#Quedarse solo con los estados en la columna destino
c2019['destino']=c2019['destino'].apply(lambda x:(re.search('(.+(?=:))', x)).group(1))
c2019.head()

,producto,calidad,presentación,origen,destino,precio mín,precio max,precio frec,obs.
Fecha,,,,,,,,,
22/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN
23/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN
24/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN
25/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN
26/04/2019,Aguacate Hass,Primera,Caja de 15 kg.,aguascalientes,nayarit,808,810,809,NaN


In [256]:
adj_2019 = c2019.groupby(['origen', 'destino']).size().rename('frecuencia').reset_index()
adj_2019.head()

,origen,destino,frecuencia
0,aguascalientes,aguascalientes,62
1,aguascalientes,chihuahua,6
2,aguascalientes,durango,24
3,aguascalientes,michoacán,12
4,aguascalientes,nayarit,18


In [257]:
#Eliminamos self-loops
adj_2019=adj_2019[adj_2019['origen']!=adj_2019['destino']]
adj_2019.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 1 to 383
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   origen      355 non-null    object
 1   destino     355 non-null    object
 2   frecuencia  355 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 11.1+ KB


In [258]:
adj_2019.head(50)

,origen,destino,frecuencia
1,aguascalientes,chihuahua,6
2,aguascalientes,durango,24
3,aguascalientes,michoacán,12
4,aguascalientes,nayarit,18
5,aguascalientes,tamaulipas,6
6,aguascalientes,zacatecas,6
8,baja california,baja california sur,84
9,baja california,coahuila,6
10,baja california,durango,6
11,baja california,jalisco,12


In [242]:
adj_2019.tail(50)

,origen,destino,frecuencia
331,tamaulipas,durango,12
332,tamaulipas,morelos,18
333,tamaulipas,nuevo león,37
334,tamaulipas,san luis potosí,12
336,tamaulipas,zacatecas,6
337,veracruz,aguascalientes,6
338,veracruz,baja california,12
339,veracruz,campeche,6
340,veracruz,chiapas,36
341,veracruz,coahuila,6


In [259]:
adj_2019['origen'].unique()

array(['aguascalientes', 'baja california', 'campeche', 'chiapas',
       'chihuahua', 'coahuila', 'colima', 'distrito federal', 'durango',
       'guanajuato', 'guerrero', 'hidalgo', 'importación', 'jalisco',
       'michoacán', 'morelos', 'méxico', 'nacional', 'nayarit',
       'nuevo león', 'oaxaca', 'puebla', 'querétaro', 'san luis potosí',
       'sinaloa', 'sonora', 'tabasco', 'tamaulipas', 'veracruz',
       'yucatán', 'zacatecas'], dtype=object)

In [260]:
adj_2019.drop(adj_2019.loc[adj_2019['origen']=='nacional'].index, inplace=True)

In [261]:
def to_id(nombre):
    if nombre == 'aguascalientes':
        return 1
    if nombre == 'baja california':
        return 2
    if nombre == 'baja california sur':
        return 3
    if nombre == 'campeche':
        return 4
    if nombre == 'chiapas': 
        return 5
    if nombre == 'chihuahua': 
        return 6
    if nombre == 'coahuila': 
        return 7
    if nombre == 'colima':
        return 8
    if nombre == 'distrito federal': 
        return 9
    if nombre == 'durango':
        return 10
    if nombre == 'guanajuato':
        return 11
    if nombre == 'guerrero':
        return 12
    if nombre == 'hidalgo': 
        return 13
    if nombre == 'jalisco':
        return 14
    if nombre == 'méxico':
        return 15
    if nombre == 'michoacán':
        return 16
    if nombre == 'morelos':
        return 17        
    if nombre == 'nayarit':
        return 18
    if nombre == 'nuevo león':
        return 19
    if nombre == 'oaxaca':
        return 20
    if nombre == 'puebla':
        return 21
    if nombre == 'querétaro':
        return 22
    if nombre == 'quintana roo':
        return 23
    if nombre == 'san luis potosí':
        return 24
    if nombre == 'sinaloa':
        return 25
    if nombre == 'sonora':
        return 26
    if nombre == 'tabasco':
        return 27
    if nombre == 'tamaulipas':
        return 28
    if nombre == 'veracruz':
        return 29
    if nombre == 'yucatán':
        return 30
    if nombre == 'zacatecas':
        return 31
    if nombre == 'importación': 
        return 35
    

In [262]:
adj_2019['origen']=adj_2019['origen'].apply(to_id)
adj_2019['destino']=adj_2019['destino'].apply(to_id)
adj_2019.head()

,origen,destino,frecuencia
1,1,6,6
2,1,10,24
3,1,16,12
4,1,18,18
5,1,28,6


In [246]:
adj_2019.tail(50)

,origen,destino,frecuencia
331,28,10,12
332,28,17,18
333,28,19,37
334,28,24,12
336,28,31,6
337,29,1,6
338,29,2,12
339,29,4,6
340,29,5,36
341,29,7,6


In [264]:
adj_2019.info()
# no hay valores nulos

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353 entries, 1 to 383
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   origen      353 non-null    int64
 1   destino     353 non-null    int64
 2   frecuencia  353 non-null    int64
dtypes: int64(3)
memory usage: 11.0 KB


In [266]:
adj_2019 = adj_2019.rename(columns={'origen':'source', 'destino':'tarjet'}, inplace=True)